# Aufgabe 10: 3-SAT Problem

In [1916]:
import numpy as np

rng = np.random.default_rng()

#Klauselmenge = rng.integers(size=(25,3), low=-20, high=21)
Klauselmenge = []
for i in range(25):
    numLiterale = rng.integers(low=1, high=4)
    if numLiterale == 1:
        Klauselmenge.append([rng.choice(np.concatenate((np.arange(-20, 0), np.arange(1, 21)))), 0, 0])
    if numLiterale == 2:
        Klauselmenge.append([rng.choice(np.concatenate((np.arange(-20, 0), np.arange(1, 21)))), rng.choice(np.concatenate((np.arange(-20, 0), np.arange(1, 21)))), 0])
    if numLiterale == 3:
        Klauselmenge.append([rng.choice(np.concatenate((np.arange(-20, 0), np.arange(1, 21)))), rng.choice(np.concatenate((np.arange(-20, 0), np.arange(1, 21)))), rng.choice(np.concatenate((np.arange(-20, 0), np.arange(1, 21))))])

print(Klauselmenge)


[[-14, 11, -13], [11, 13, 0], [-10, 6, 11], [14, 0, 0], [14, 15, 0], [17, -7, 0], [-8, -11, -11], [20, -15, 0], [14, 16, 0], [4, 0, 0], [7, 0, 0], [-19, 9, 11], [15, -16, 0], [2, -12, 0], [-19, -13, -4], [4, -6, 2], [-18, 14, 0], [-6, 15, 0], [-1, 0, 0], [-16, 0, 0], [-13, -18, -1], [10, 0, 0], [-19, -9, 3], [3, 2, -20], [-5, 0, 0]]


In [1917]:
def InitPopulation(numIndividuen, numVariablen):
    initPopulation = rng.integers(size=(numIndividuen, numVariablen + 1), low=0, high=2)
    for individuum in initPopulation:
        individuum[0] = 1
    return initPopulation

In [1918]:
startPopulation = InitPopulation(100, 20)
print(startPopulation)

[[1 0 0 ... 0 1 1]
 [1 0 0 ... 0 1 1]
 [1 0 1 ... 0 1 1]
 ...
 [1 1 0 ... 0 1 0]
 [1 1 0 ... 0 0 1]
 [1 0 1 ... 0 1 1]]


In [1919]:
def sortPopulation(population):
    bewertung = {}
    for i, individuum in enumerate(population):
        erfüllteKlauseln = 0
        for klausel in Klauselmenge:
            for variable in klausel:
                if variable > 0 and individuum[variable] == 1:
                    erfüllteKlauseln = erfüllteKlauseln + 1
                    break
                elif variable < 0 and individuum[-1 * variable] == 0:
                    erfüllteKlauseln = erfüllteKlauseln + 1
                    break
        bewertung[i] = erfüllteKlauseln

    sortierteIndividuen = dict(sorted(bewertung.items(), key=lambda item: item[1]))

    return sortierteIndividuen


In [1920]:
anzahlSelection = 20
def Selektion(sortedPopulation, population):
    selected = []
    np.array(selected)
    for i in range(anzahlSelection):
        individuum = population[list(sortedPopulation.keys())[99-i]]
        #print(individuum)
        selected.extend(individuum)
        
        #selected = np.concatenate((selected,individuum ), axis=0)
        #selected[99-i] = [individuum]
        #selectet = selected + individuum
        #selected
    selected = np.reshape(selected, (anzahlSelection, 21))
    #print(selected)
    return selected

    

In [1921]:
def Mutation(sortedPopulation, population):
    mutatedPopulation = []
    for i in range(20):
        individuum = population[list(sortedPopulation.keys())[99-i] ]
        mutationSpot = rng.integers(low=1, high=21)
        individuum[mutationSpot] = 1 if individuum[mutationSpot] == 0 else 0
        mutatedPopulation.append(individuum)

    return mutatedPopulation
        

In [1922]:
def Rekombination(sortedPopulation, population):
    recombinedPopulation = []
    for i in range(15):
        individuum1 = np.array_split(population[list(sortedPopulation.keys())[99-i]], 2)
        individuum2 = np.array_split(population[list(sortedPopulation.keys())[rng.integers(low=1, high=21)]], 2)

        new1 = np.concatenate((individuum1[0], individuum2[1]))
        new2 = np.concatenate((individuum2[0], individuum1[1]))

        recombinedPopulation.append(new1)
        recombinedPopulation.append(new2)
        
    return recombinedPopulation


In [1923]:
def createNewRandom():
    newRandoms = rng.integers(size=(30, 20 + 1), low=0, high=2)
    for individuum in newRandoms:
        individuum[0] = 1
        
    return newRandoms

In [1924]:
def Evolution(iterations, startPopulation):
    population = startPopulation
    for i in range(iterations):
        sortedPopulation = sortPopulation(population)

        print(list(sortedPopulation.values())[99])

        #print(sortedPopulation)

        selektion = Selektion(sortedPopulation, population)
        mutation = Mutation(sortedPopulation, population)
        rekombination = Rekombination(sortedPopulation, population)
        newRandoms = createNewRandom()

        newPopulation = np.concatenate((selektion, newRandoms,mutation,rekombination ),  axis=0)
        #print(newPopulation)
        #print(selektion)
        #print(len(newPopulation))
        
        #print(newRandoms)
        #print(newPopulation)

        population = newPopulation

        
        

        
        
Evolution(20, startPopulation)

23
23
24
24
24
24
24
25
25
25
25
25
25
25
25
25
25
25
25
25
